In [1]:
import json
import re
import matplotlib.pyplot as plt
from collections import Counter
import random

In [2]:
train_set = json.load(open('train.json'))
dev_set = json.load(open('dev.json'))
test_set = json.load(open('test.json'))
data_all = train_set | dev_set | test_set

In [3]:
for image, annotations in data_all.items():
    frame_keys = None
    for frame in annotations['frames']:
        if frame_keys is None: frame_keys = set(frame.keys())
        else: assert frame_keys == set(frame.keys())

In [4]:
no_agreement_nouns, no_agreement_images = 0, 0
total_nouns, total_images = 0, len(data_all)
for image, annotations in data_all.items():
    no_agreement_image = False
    frame_collapsed = {k: [] for k in annotations['frames'][0].keys()}
    total_nouns += len(frame_collapsed)
    for frame in annotations['frames']:
        for k, v in frame.items():
            frame_collapsed[k].append(v)
    for k in frame_collapsed.keys():
        if Counter(frame_collapsed[k]).most_common(1)[0][1] == 1:
            no_agreement_nouns += 1
            no_agreement_image = True
        frame_collapsed[k] = Counter(frame_collapsed[k]).most_common(1)[0][0]
    no_agreement_images += no_agreement_image

In [5]:
no_agreement_nouns, total_nouns, no_agreement_nouns / total_nouns

(105857, 451916, 0.23424043406296746)

In [6]:
no_agreement_images, total_images, no_agreement_images / total_images

(73618, 126102, 0.5837972435012926)

In [8]:
data_all_collapsed = {}
for image, annotations in data_all.items():
    frame_collapsed = {k: [] for k in annotations['frames'][0].keys()}
    for frame in annotations['frames']:
        for k, v in frame.items():
            frame_collapsed[k].append(v)
    for k in frame_collapsed.keys():
        maj_v, count = Counter(frame_collapsed[k]).most_common(1)[0]
        if count == 1:
            frame_collapsed[k] = random.choice(frame_collapsed[k])
        else:
            frame_collapsed[k] = maj_v
    data_all_collapsed[image] = {'verb': annotations['verb'], 'frames': [frame_collapsed]}